In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [2]:
#To display all the columns and rows
pd.set_option('display.max_columns', 1000);
pd.set_option('display.max_rows', 1000);
pd.set_option('display.width', 1000);
#To ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#To supress scientific notations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
#Loading the file
merged_labs = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/merged_labs.csv')

/tmp/ipython-input-3-53828618.py:2: DtypeWarning: Columns (7,16) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_labs = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/merged_labs.csv')


In [4]:
merged_labs.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56917 entries, 0 to 56916
Data columns (total 2389 columns):
 #     Column                                                  Non-Null Count  Dtype  
---    ------                                                  --------------  -----  
 0     subject_id                                              56917 non-null  int64  
 1     hadm_id                                                 56917 non-null  int64  
 2     procedures_applied                                      30277 non-null  object 
 3     icd_code                                                56917 non-null  object 
 4     long_title                                              56917 non-null  object 
 5     seq_num                                                 56917 non-null  int64  
 6     Classification                                          56917 non-null  object 
 7     included                                                56917 non-null  object 
 8     class       

In [5]:
#Filling up as this might be used later for other classification or regression tasks
merged_labs['time_to_diagnosis'] = merged_labs['time_to_diagnosis'].fillna(0)
merged_labs['days_to_next_admission'] = merged_labs['days_to_next_admission'].fillna(-1)

In [6]:
#Creating a copy of the main dataset
df = merged_labs.copy()

In [7]:
df.shape

(56917, 2389)

##1. Data Cleaning

In [8]:
#Step 1 : Removing all the columns that have more than 75% data missing. This is data cleaning based on missing data.
# Identify columns with more than 70% missing values
threshold = 0.7
df_cleaned = df.loc[:, df.isnull().mean() <= threshold]

# Optional: Check how many columns were dropped
dropped_cols = df.shape[1] - df_cleaned.shape[1]
print(f"Columns dropped: {dropped_cols}")
print(f"New shape: {df_cleaned.shape}")

Columns dropped: 1495
New shape: (56917, 894)


In [9]:
df_cleaned.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56917 entries, 0 to 56916
Data columns (total 894 columns):
 #    Column                                       Non-Null Count  Dtype  
---   ------                                       --------------  -----  
 0    subject_id                                   56917 non-null  int64  
 1    hadm_id                                      56917 non-null  int64  
 2    procedures_applied                           30277 non-null  object 
 3    icd_code                                     56917 non-null  object 
 4    long_title                                   56917 non-null  object 
 5    seq_num                                      56917 non-null  int64  
 6    Classification                               56917 non-null  object 
 7    included                                     56917 non-null  object 
 8    class                                        56917 non-null  int64  
 9    gender                                       56917 non-null

The first_value_<lab> typically represents the initial lab value at admission or early in the encounter. This is often the most unbiased indicator of a patient's state before diagnosis or intervention, which is crucial for prediction models with early diagnosis. Many lab values (max, min, last, etc.) may exist only in a small subset of patients, and without the first_value_, there's no anchor to interpret the data. Dropping them improves signal-to-noise ratio and reduces model overfitting. With over 2,000 columns, the dataset is at risk of the curse of dimensionality. By using a rule-based selection (i.e., “keep only lab features if first_value_ exists”), you reduce dimensions intelligently, not arbitrarily.

In [10]:
#Step 2: Keeping only those lab records for which we have first_value_<labs>. This is data cleaning based on data's structure.

#Rename columns 318 to 893 with frequency_value_ prefix
count_cols = df_cleaned.columns[318:893]
rename_map = {col: f"frequency_value_{col}" for col in count_cols}
df_cleaned = df_cleaned.rename(columns=rename_map)

#Standardize frequency_value_ column names: lowercase + replace spaces with underscores
df_cleaned.columns = [
    col.lower().replace(" ", "_") if col.startswith("frequency_value_") else col
    for col in df_cleaned.columns
]

#Identify first_value_ columns and extract base lab names
first_value_cols = [col for col in df_cleaned.columns if col.startswith('first_value_')]
valid_labs = [col.replace('first_value_', '') for col in first_value_cols]
#Collect lab-related columns to keep
allowed_lab_cols = set(first_value_cols)
for lab in valid_labs:
    for prefix in ['last_value_', 'max_value_', 'min_value_', 'frequency_value_']:
        colname = f"{prefix}{lab}"
        if colname in df_cleaned.columns:
            allowed_lab_cols.add(colname)

#Preserve original column order
final_cols = []
for col in df_cleaned.columns:
    if not re.match(r'(first|last|max|min|frequency)_value_', col):  # non-lab column
        final_cols.append(col)
    elif col in allowed_lab_cols:
        final_cols.append(col)

df_cleaned_1 = df_cleaned[final_cols]

#Check on dropped columns
dropped = df_cleaned.shape[1] - df_cleaned_1.shape[1]
print(f"Dropped {dropped} columns. Final shape: {df_cleaned_1.shape}")

Dropped 529 columns. Final shape: (56917, 365)


In [11]:
df_cleaned_1.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56917 entries, 0 to 56916
Data columns (total 365 columns):
 #    Column                                           Non-Null Count  Dtype  
---   ------                                           --------------  -----  
 0    subject_id                                       56917 non-null  int64  
 1    hadm_id                                          56917 non-null  int64  
 2    procedures_applied                               30277 non-null  object 
 3    icd_code                                         56917 non-null  object 
 4    long_title                                       56917 non-null  object 
 5    seq_num                                          56917 non-null  int64  
 6    Classification                                   56917 non-null  object 
 7    included                                         56917 non-null  object 
 8    class                                            56917 non-null  int64  
 9    gender         

In [12]:
df_cleaned_1.shape

(56917, 365)

##2. Feature Extraction

In [13]:
#Extracting time to first labs and time to first abnormal labs compared to admittime.
#Creating a copy of df_cleaned_1 to avoid modifying it directly
df_cleaned_2 = df_cleaned_1.copy()
#Ensure datetime columns are in correct format
df_cleaned_2['first_lab_time'] = pd.to_datetime(df_cleaned_2['first_lab_time'], errors='coerce')
df_cleaned_2['first_abnormal_lab_time'] = pd.to_datetime(df_cleaned_2['first_abnormal_lab_time'], errors='coerce')
df_cleaned_2['admittime'] = pd.to_datetime(df_cleaned_2['admittime'], errors='coerce')
#Compute time to first lab in hours
df_cleaned_2['time_to_first_lab_hrs'] = ((df_cleaned_2['first_lab_time'] - df_cleaned_2['admittime']).dt.total_seconds() / 3600)
#Compute time to first abnormal lab in hours
df_cleaned_2['time_to_first_abnormal_lab_hrs'] = ((df_cleaned_2['first_abnormal_lab_time'] - df_cleaned_2['admittime']).dt.total_seconds() / 3600)

In [14]:
#Splitting anchor_year_group into two columns
if 'anchor_year_group' in df_cleaned_2.columns:
    # Split 'anchor_year_group' into two new columns
    df_cleaned_2[['anchor_year_start', 'anchor_year_end']] = (
        df_cleaned_2['anchor_year_group'].str.split(' - ', expand=True).astype(int)
    )
# Drop the original column
df_cleaned_2.drop(columns='anchor_year_group', inplace=True)

In [15]:
#Splitting phenotype_flags into
#Extract all unique flags from df_cleaned_2['phenotype_flags']
all_flags = set()
for flags in df_cleaned_2['phenotype_flags'].dropna():
    for flag in flags.split(', '):
        all_flags.add(flag)
all_flags = sorted(all_flags)
#Create binary columns for each flag
for flag in all_flags:
    df_cleaned_2[flag] = df_cleaned_2['phenotype_flags'].apply(
        lambda x: 1 if pd.notnull(x) and flag in x else 0
)
#Drop the original column
df_cleaned_2.drop(columns=['phenotype_flags','normal'], inplace=True)

In [16]:
# List of duration columns
duration_cols = ['total_med_duration', 'avg_med_duration', 'max_med_duration']
for col in duration_cols:
    # Convert to timedelta
    df_cleaned_2[col] = pd.to_timedelta(df_cleaned_2[col], errors='coerce')

    # Convert to float (e.g., total hours)
    df_cleaned_2[col + '_hrs'] = df_cleaned_2[col].dt.total_seconds() / 3600

#Dropping the original columns
df_cleaned_2.drop(columns=duration_cols, inplace=True)

In [17]:
df_cleaned_2.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56917 entries, 0 to 56916
Data columns (total 373 columns):
 #    Column                                           Non-Null Count  Dtype         
---   ------                                           --------------  -----         
 0    subject_id                                       56917 non-null  int64         
 1    hadm_id                                          56917 non-null  int64         
 2    procedures_applied                               30277 non-null  object        
 3    icd_code                                         56917 non-null  object        
 4    long_title                                       56917 non-null  object        
 5    seq_num                                          56917 non-null  int64         
 6    Classification                                   56917 non-null  object        
 7    included                                         56917 non-null  object        
 8    class                   

In [18]:
df_cleaned_2.head()

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,first_culture_time,spec_types,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,medications_given,prn_med_count,first_med_time,num_prescriptions_drugs,num_unique_prescriptions_drugs,drug_list,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_lab_time,first_abnormal_lab_time,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_alanine_aminotransferase_(alt),first_value_albumin,first_value_alkaline_phosphatase,first_value_anion_gap,first_value_asparate_aminotransferase_(ast),first_value_basophils,first_value_bicarbonate,"first_value_bilirubin,_total","first_value_calcium,_total",first_value_chloride,first_value_creatinine,first_value_eosinophils,first_value_glucose,first_value_h,first_value_hematocrit,first_value_hemoglobin,first_value_i,first_value_inr(pt),first_value_immature_granulocytes,first_value_l,first_value_lactate,first_value_lymphocytes,first_value_mch,first_value_mchc,first_value_mcv,first_value_magnesium,first_value_monocytes,first_value_neutrophils,first_value_pt,first_value_ptt,first_value_phosphate,first_value_platelet_count,first_value_potassium,first_value_rdw,first_value_rdw-sd,first_value_red_blood_cells,first_value_sodium,first_value_specific_gravity,first_value_urea_nitrogen,first_value_white_blood_cells,first_value_ph,max_value_absolute_basophil_count,max_value_absolute_eosinophil_count,max_value_absolute_lymphocyte_count,max_value_absolute_monocyte_count,max_value_absolute_neutrophil_count,max_value_alanine_aminotransferase_(alt),max_value_albumin,max_value_alkaline_phosphatase,max_value_anion_gap,max_value_asparate_aminotransferase_(ast),max_value_basophils,max_value_bicarbonate,"max_value_bilirubin,_total","max_value_calcium,_total",max_value_chloride,max_value_creatinine,max_value_eosinophils,max_value_glucose,max_value_h,max_value_hematocrit,max_value_hemoglobin,max_value_i,max_value_inr(pt),max_value_immature_granulocytes,max_value_l,max_value_lactate,max_value_lymphocytes,max_value_mch,max_value_mchc,max_value_mcv,max_value_magnesium,max_value_monocytes,max_value_neutrophils,max_value_pt,max_value_ptt,max_value_phosphate,max_value_platelet_count,max_valu

##3. Data de-duplication

In [19]:
print(df_cleaned_2.duplicated().any())

False


In [20]:
print(df_cleaned_2['subject_id'].duplicated().any())

False


##4. Feature Encoding

In [21]:
df_cleaned_2.select_dtypes(exclude = ['int64','int32','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56917 entries, 0 to 56916
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   procedures_applied          30277 non-null  object        
 1   icd_code                    56917 non-null  object        
 2   long_title                  56917 non-null  object        
 3   Classification              56917 non-null  object        
 4   included                    56917 non-null  object        
 5   gender                      56917 non-null  object        
 6   admittime                   56917 non-null  datetime64[ns]
 7   dischtime                   56917 non-null  object        
 8   admission_type              56917 non-null  object        
 9   admission_location          56917 non-null  object        
 10  discharge_location          43991 non-null  object        
 11  insurance                   56917 non-null  object    

In [22]:
categorical_cols = ['gender', 'admission_type', 'admission_location','discharge_location', 'insurance', 'language', 'marital_status', 'race',
                    'admission_season', 'first_careunit', 'last_careunit']

In [23]:
mappings = {
    'gender': {
        'M': 1,
        'F': 2
    },
    'admission_type': {
        'OBSERVATION ADMIT': 1,
        'EW EMER.': 2,
        'EU OBSERVATION': 3,
        'URGENT': 4,
        'DIRECT OBSERVATION': 5,
        'SURGICAL SAME DAY ADMISSION': 6,
        'DIRECT EMER.': 7,
        'ELECTIVE': 8,
        'AMBULATORY OBSERVATION': 9
    },
    'admission_location': {
        'EMERGENCY ROOM': 1,
        'PHYSICIAN REFERRAL': 2,
        'TRANSFER FROM HOSPITAL': 3,
        'WALK-IN/SELF REFERRAL': 4,
        'CLINIC REFERRAL': 5,
        'INTERNAL TRANSFER TO OR FROM PSYCH': 6,
        'TRANSFER FROM SKILLED NURSING FACILITY': 7,
        'AMBULATORY SURGERY TRANSFER': 8,
        'PROCEDURE SITE': 9,
        'PACU': 10,
        'INFORMATION NOT AVAILABLE': 11
    },
    'discharge_location': {
        'HOME': 1,
        'HOME HEALTH CARE': 2,
        'SKILLED NURSING FACILITY': 3,
        'DIED': 4,
        'REHAB': 5,
        'HOSPICE': 6,
        'CHRONIC/LONG TERM ACUTE CARE': 7,
        'AGAINST ADVICE': 8,
        'ACUTE HOSPITAL': 9,
        'PSYCH FACILITY': 10,
        'OTHER FACILITY': 11,
        'ASSISTED LIVING': 12,
        'HEALTHCARE FACILITY': 13,
        np.nan: 14,
        },
    'insurance': {
        'Medicare': 1,
        'Medicaid': 2,
        'Other': 3
    },
    'language': {
        'ENGLISH': 1,
        '?': 2
    },
    'marital_status': {
        'MARRIED': 1,
        'SINGLE': 2,
        'DIVORCED': 3,
        'WIDOWED': 4,
        np.nan: 5
    },
    'race' : {
    'WHITE': 1,
    'WHITE - OTHER EUROPEAN': 1,
    'WHITE - EASTERN EUROPEAN': 1,
    'WHITE - RUSSIAN': 1,
    'WHITE - BRAZILIAN': 1,

    'BLACK/AFRICAN AMERICAN': 2,
    'BLACK/AFRICAN': 2,
    'BLACK/CARIBBEAN ISLAND': 2,
    'BLACK/CAPE VERDEAN': 2,

    'ASIAN': 3,
    'ASIAN - CHINESE': 3,
    'ASIAN - ASIAN INDIAN': 3,
    'ASIAN - KOREAN': 3,
    'ASIAN - SOUTH EAST ASIAN' : 3,

    'HISPANIC/LATINO - PUERTO RICAN': 4,
    'HISPANIC/LATINO - CUBAN': 4,
    'HISPANIC OR LATINO': 4,
    'HISPANIC/LATINO - SALVADORAN': 4,
    'HISPANIC/LATINO - GUATEMALAN': 4,
    'HISPANIC/LATINO - HONDURAN': 4,
    'HISPANIC/LATINO - DOMINICAN': 4,
    'HISPANIC/LATINO - MEXICAN': 4,
    'HISPANIC/LATINO - CENTRAL AMERICAN': 4,
    'HISPANIC/LATINO - COLUMBIAN': 4,
    'SOUTH AMERICAN': 4,

    'PORTUGUESE': 5,
    'OTHER': 5,
    'AMERICAN INDIAN/ALASKA NATIVE': 5,
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 5,
    'MULTIPLE RACE/ETHNICITY':5,

    'UNKNOWN': 6,
    'UNABLE TO OBTAIN': 6,
    'PATIENT DECLINED TO ANSWER': 6,
    },

    'admission_season': {
        'Winter': 1,
        'Spring': 2,
        'Summer': 3,
        'Fall': 4
    },
    'first_careunit' : {
    'Emergency Department': 1,
    'Discharge Lounge': 2,
    'Medicine': 3,
    'Med/Surg': 3,
    'Med/Surg/GYN': 3,
    'Med/Surg/Trauma': 3,
    'Hematology/Oncology Intermediate': 4,
    'Hematology/Oncology': 4,
    'Medicine/Cardiology': 5,
    'Medicine/Cardiology Intermediate': 5,
    'Psychiatry': 6,
    'Surgery': 7,
    'Surgery/Trauma': 7,
    'Transplant': 8,
    'Neurology': 9,
    'Neuro Intermediate': 9,
    'Neuro Stepdown': 9,
    'Cardiology': 10,
    'Cardiology Surgery Intermediate': 10,
    'Cardiac Surgery': 10,
    'Vascular': 11,
    'Labor & Delivery': 12,
    'Obstetrics Antepartum': 12,
    'Obstetrics Postpartum': 12,
    'Obstetrics (Postpartum & Antepartum)': 12,
    'Observation': 13,
    'Emergency Department Observation': 13,

    # Group all ICU-like care units under category 14
    'Medical/Surgical Intensive Care Unit (MICU/SICU)': 14,
    'Trauma SICU (TSICU)': 14,
    'Cardiac Vascular Intensive Care Unit (CVICU)': 14,
    'Medical Intensive Care Unit (MICU)': 14,
    'Neuro Surgical Intensive Care Unit (Neuro SICU)': 14,
    'Surgical Intensive Care Unit (SICU)': 14,
    'Coronary Care Unit (CCU)': 14,
    'PACU': 15,
    'Unknown': 16

    },

    'last_careunit' : {
    'Medicine': 3,

    'Hematology/Oncology Intermediate': 4,
    'Hematology/Oncology': 4,

    'Medicine/Cardiology': 5,
    'Medicine/Cardiology Intermediate': 5,

    'Psychiatry': 6,

    'Surgery': 7,
    'Med/Surg': 7,
    'Surgery/Trauma': 7,
    'Med/Surg/Trauma': 7,

    'Transplant': 8,

    'Neurology': 9,
    'Neuro Stepdown': 9,
    'Neuro Intermediate': 9,

    'Cardiology': 10,
    'Cardiology Surgery Intermediate': 10,
    'Cardiac Surgery': 10,

    'Vascular': 11,

    'Labor & Delivery': 12,
    'Med/Surg/GYN': 12,
    'Obstetrics Antepartum': 12,
    'Obstetrics Postpartum': 12,
    'Obstetrics (Postpartum & Antepartum)': 12,

    'Observation': 13,
    'Emergency Department Observation': 13,

    # ICU-type units grouped under 14
    'Medical/Surgical Intensive Care Unit (MICU/SICU)': 14,
    'Medical Intensive Care Unit (MICU)': 14,
    'Surgical Intensive Care Unit (SICU)': 14,
    'Trauma SICU (TSICU)': 14,
    'Cardiac Vascular Intensive Care Unit (CVICU)': 14,
    'Coronary Care Unit (CCU)': 14,
    'Neuro Surgical Intensive Care Unit (Neuro SICU)': 14,

    'PACU': 15,  # Post Anesthesia Care Unit — mapped separately
    'Unknown': 16  # Just in case
    }

}

In [24]:
#Create a copy to preserve df_cleaned_2
df_cleaned_3 = df_cleaned_2.copy()

#Apply the mapping to appropriate columns
for col, mapping in mappings.items():
    if col in df_cleaned_3.columns:
        df_cleaned_3[col] = df_cleaned_3[col].map(mapping).fillna(-1).astype(int)

In [25]:
df_cleaned_2[df_cleaned_3['race']==-1]

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,first_culture_time,spec_types,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,medications_given,prn_med_count,first_med_time,num_prescriptions_drugs,num_unique_prescriptions_drugs,drug_list,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_lab_time,first_abnormal_lab_time,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_alanine_aminotransferase_(alt),first_value_albumin,first_value_alkaline_phosphatase,first_value_anion_gap,first_value_asparate_aminotransferase_(ast),first_value_basophils,first_value_bicarbonate,"first_value_bilirubin,_total","first_value_calcium,_total",first_value_chloride,first_value_creatinine,first_value_eosinophils,first_value_glucose,first_value_h,first_value_hematocrit,first_value_hemoglobin,first_value_i,first_value_inr(pt),first_value_immature_granulocytes,first_value_l,first_value_lactate,first_value_lymphocytes,first_value_mch,first_value_mchc,first_value_mcv,first_value_magnesium,first_value_monocytes,first_value_neutrophils,first_value_pt,first_value_ptt,first_value_phosphate,first_value_platelet_count,first_value_potassium,first_value_rdw,first_value_rdw-sd,first_value_red_blood_cells,first_value_sodium,first_value_specific_gravity,first_value_urea_nitrogen,first_value_white_blood_cells,first_value_ph,max_value_absolute_basophil_count,max_value_absolute_eosinophil_count,max_value_absolute_lymphocyte_count,max_value_absolute_monocyte_count,max_value_absolute_neutrophil_count,max_value_alanine_aminotransferase_(alt),max_value_albumin,max_value_alkaline_phosphatase,max_value_anion_gap,max_value_asparate_aminotransferase_(ast),max_value_basophils,max_value_bicarbonate,"max_value_bilirubin,_total","max_value_calcium,_total",max_value_chloride,max_value_creatinine,max_value_eosinophils,max_value_glucose,max_value_h,max_value_hematocrit,max_value_hemoglobin,max_value_i,max_value_inr(pt),max_value_immature_granulocytes,max_value_l,max_value_lactate,max_value_lymphocytes,max_value_mch,max_value_mchc,max_value_mcv,max_value_magnesium,max_value_monocytes,max_value_neutrophils,max_value_pt,max_value_ptt,max_value_phosphate,max_value_platelet_count,max_valu

In [26]:
# Checking the mappings
#'gender', 'admission_type', 'admission_location','discharge_location', 'insurance', 'language', 'marital_status', 'race',
#'first_careunit', 'last_careunit', 'admission_season'
df_cleaned_3['race'].value_counts()

,count
race,
1,37985
2,7992
6,3739
4,2605
5,2518
3,2078


In [27]:
df_cleaned_3.shape

(56917, 373)

##5. Label Creation

In [28]:
# Create the target column and save to new DataFrame
df_cleaned_4 = df_cleaned_3.copy()
df_cleaned_4['target'] = df_cleaned_4['class'].apply(lambda x: 1 if x == 1 else 0)

In [29]:
df_cleaned_4.shape

(56917, 374)

In [30]:
#Using this file separately for EDA
df_cleaned_4.to_csv('/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/eda_disorders.csv', index=False, header=True)

##6. Negative Data Check

In [31]:
df_cleaned_5 = df_cleaned_4.copy()

In [32]:
#Select numeric columns
numeric_cols = df_cleaned_5.select_dtypes(include=[np.number])
# Find negative values
negatives = (numeric_cols < 0).sum()
negatives = negatives[negatives > 0]  # Only keep columns with negatives
# Print or view the result
print("Columns with negative values:")
print(negatives)

Columns with negative values:
days_to_next_admission            35703
los                                  16
time_to_emergency_hrs             39166
avg_duration_days                    62
max_duration_days                     5
time_to_first_med_hrs              5079
first_value_anion_gap                 1
min_value_anion_gap                  10
time_to_first_lab_hrs              8340
time_to_first_abnormal_lab_hrs     8183
total_med_duration_hrs               65
avg_med_duration_hrs                 65
max_med_duration_hrs                  5
dtype: int64


In [33]:
#Checking the columns one by one.
#1. LOS
df_cleaned_5[df_cleaned_5['los'] < 0]

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,first_culture_time,spec_types,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,medications_given,prn_med_count,first_med_time,num_prescriptions_drugs,num_unique_prescriptions_drugs,drug_list,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_lab_time,first_abnormal_lab_time,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_alanine_aminotransferase_(alt),first_value_albumin,first_value_alkaline_phosphatase,first_value_anion_gap,first_value_asparate_aminotransferase_(ast),first_value_basophils,first_value_bicarbonate,"first_value_bilirubin,_total","first_value_calcium,_total",first_value_chloride,first_value_creatinine,first_value_eosinophils,first_value_glucose,first_value_h,first_value_hematocrit,first_value_hemoglobin,first_value_i,first_value_inr(pt),first_value_immature_granulocytes,first_value_l,first_value_lactate,first_value_lymphocytes,first_value_mch,first_value_mchc,first_value_mcv,first_value_magnesium,first_value_monocytes,first_value_neutrophils,first_value_pt,first_value_ptt,first_value_phosphate,first_value_platelet_count,first_value_potassium,first_value_rdw,first_value_rdw-sd,first_value_red_blood_cells,first_value_sodium,first_value_specific_gravity,first_value_urea_nitrogen,first_value_white_blood_cells,first_value_ph,max_value_absolute_basophil_count,max_value_absolute_eosinophil_count,max_value_absolute_lymphocyte_count,max_value_absolute_monocyte_count,max_value_absolute_neutrophil_count,max_value_alanine_aminotransferase_(alt),max_value_albumin,max_value_alkaline_phosphatase,max_value_anion_gap,max_value_asparate_aminotransferase_(ast),max_value_basophils,max_value_bicarbonate,"max_value_bilirubin,_total","max_value_calcium,_total",max_value_chloride,max_value_creatinine,max_value_eosinophils,max_value_glucose,max_value_h,max_value_hematocrit,max_value_hemoglobin,max_value_i,max_value_inr(pt),max_value_immature_granulocytes,max_value_l,max_value_lactate,max_value_lymphocytes,max_value_mch,max_value_mchc,max_value_mcv,max_value_magnesium,max_value_monocytes,max_value_neutrophils,max_value_pt,max_value_ptt,max_value_phosphate,max_value_platelet_count,max_valu

In [34]:
#Make sure admittime and dischtime are datetime
df_cleaned_5['admittime'] = pd.to_datetime(df_cleaned_5['admittime'])
df_cleaned_5['dischtime'] = pd.to_datetime(df_cleaned_5['dischtime'])

# Identify rows where dischtime is exactly at midnight (00:00:00)
midnight_mask = df_cleaned_5['dischtime'].dt.time == pd.to_datetime('00:00:00').time()
# Correct those dischtimes to 23:59:00 of the same date
df_cleaned_5.loc[midnight_mask, 'dischtime'] = df_cleaned_5.loc[midnight_mask, 'dischtime'] + pd.Timedelta(hours=23, minutes=59)
# Recalculate LOS (length of stay) in days
df_cleaned_5['los'] = (df_cleaned_5['dischtime'] - df_cleaned_5['admittime']).dt.total_seconds() / (24 * 3600)
# Sanity check: show any remaining negative LOS
print("Remaining rows with negative LOS:", (df_cleaned_5['los'] < 0).sum())

Remaining rows with negative LOS: 6


In [35]:
df_cleaned_5[df_cleaned_5['los'] < 0]

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,first_culture_time,spec_types,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,medications_given,prn_med_count,first_med_time,num_prescriptions_drugs,num_unique_prescriptions_drugs,drug_list,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_lab_time,first_abnormal_lab_time,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_alanine_aminotransferase_(alt),first_value_albumin,first_value_alkaline_phosphatase,first_value_anion_gap,first_value_asparate_aminotransferase_(ast),first_value_basophils,first_value_bicarbonate,"first_value_bilirubin,_total","first_value_calcium,_total",first_value_chloride,first_value_creatinine,first_value_eosinophils,first_value_glucose,first_value_h,first_value_hematocrit,first_value_hemoglobin,first_value_i,first_value_inr(pt),first_value_immature_granulocytes,first_value_l,first_value_lactate,first_value_lymphocytes,first_value_mch,first_value_mchc,first_value_mcv,first_value_magnesium,first_value_monocytes,first_value_neutrophils,first_value_pt,first_value_ptt,first_value_phosphate,first_value_platelet_count,first_value_potassium,first_value_rdw,first_value_rdw-sd,first_value_red_blood_cells,first_value_sodium,first_value_specific_gravity,first_value_urea_nitrogen,first_value_white_blood_cells,first_value_ph,max_value_absolute_basophil_count,max_value_absolute_eosinophil_count,max_value_absolute_lymphocyte_count,max_value_absolute_monocyte_count,max_value_absolute_neutrophil_count,max_value_alanine_aminotransferase_(alt),max_value_albumin,max_value_alkaline_phosphatase,max_value_anion_gap,max_value_asparate_aminotransferase_(ast),max_value_basophils,max_value_bicarbonate,"max_value_bilirubin,_total","max_value_calcium,_total",max_value_chloride,max_value_creatinine,max_value_eosinophils,max_value_glucose,max_value_h,max_value_hematocrit,max_value_hemoglobin,max_value_i,max_value_inr(pt),max_value_immature_granulocytes,max_value_l,max_value_lactate,max_value_lymphocytes,max_value_mch,max_value_mchc,max_value_mcv,max_value_magnesium,max_value_monocytes,max_value_neutrophils,max_value_pt,max_value_ptt,max_value_phosphate,max_value_platelet_count,max_valu

Here, we can see that edouttime has simply been mentioned as dischtime, which is greater than admittime. We excluded records where the discharge time occurred after the formal inpatient admission time. This scenario is clinically implausible and typically arises from data entry errors, system-level logging inconsistencies, or the reuse of ED discharge timestamps as inpatient discharge placeholders. Including such records would introduce noise and reduce the reliability of derived features like length of stay, admission duration, and care transitions. By excluding these entries, we aim to preserve the temporal integrity of the clinical care timeline for downstream machine learning tasks.

In [36]:
#Removing rows with negative los
df_cleaned_5 = df_cleaned_5[df_cleaned_5['los'] >= 0].copy()

In [37]:
#Select numeric columns
numeric_cols = df_cleaned_5.select_dtypes(include=[np.number])
# Find negative values
negatives = (numeric_cols < 0).sum()
negatives = negatives[negatives > 0]  # Only keep columns with negatives
# Print or view the result
print("Columns with negative values:")
print(negatives)

Columns with negative values:
days_to_next_admission            35698
time_to_emergency_hrs             39160
avg_duration_days                    62
max_duration_days                     5
time_to_first_med_hrs              5079
first_value_anion_gap                 1
min_value_anion_gap                  10
time_to_first_lab_hrs              8340
time_to_first_abnormal_lab_hrs     8183
total_med_duration_hrs               65
avg_med_duration_hrs                 65
max_med_duration_hrs                  5
dtype: int64


The `time_to_emergency_hrs` column retains the directionality of patient flow, distinguishing whether the emergency visit occurred before or after admission, which is clinically meaningful. Preserving this signed information allows models to capture important temporal patterns related to patient deterioration or admission pathways. Columns, time_to_first_med_hrs, time_to_first_lab_hrs, or time_to_first_abnormal_lab_hrs can be clinically valid, indicating that the event (first medication, lab test, or abnormal lab) happened before formal admission. This can happen if, for example, a patient was initially seen as outpatient or in the ED, got labs or meds, and was admitted afterward due to worsening condition. So no change is required even in these columns.

In [38]:
#Columns to exclude
exclude_cols = ['time_to_first_med_hrs','time_to_first_lab_hrs','time_to_first_abnormal_lab_hrs','time_to_emergency_hrs']
#Find negative counts excluding specified columns
negatives = (numeric_cols < 0).sum()
negatives = negatives[(negatives > 0) & (~negatives.index.isin(exclude_cols))]
print("Columns with negative values excluding the 4 specified columns:")
print(negatives)

Columns with negative values excluding the 4 specified columns:
days_to_next_admission    35698
avg_duration_days            62
max_duration_days             5
first_value_anion_gap         1
min_value_anion_gap          10
total_med_duration_hrs       65
avg_med_duration_hrs         65
max_med_duration_hrs          5
dtype: int64


In [39]:
#Columns which still have negative data
cols_to_check = ['avg_duration_days','max_duration_days','first_value_anion_gap','min_value_anion_gap','total_med_duration_hrs',
                 'avg_med_duration_hrs', 'max_med_duration_hrs']
# Boolean mask for any negative value in these columns
neg_mask = (df_cleaned_5[cols_to_check] < 0).any(axis=1)
# Get the value counts of 'class' for rows with negatives
class_counts = df_cleaned_5.loc[neg_mask, 'class'].value_counts()

In [40]:
class_counts

,count
class,
3,33
4,32
2,6
5,4
1,1


Since, only 1 rare disease row is there we can drop these rows.

In [41]:
# Drop those rows inplace
df_cleaned_5 = df_cleaned_5.loc[~neg_mask].copy()
print(f"Dropped {neg_mask.sum()} rows with negative values in specified columns.")

Dropped 76 rows with negative values in specified columns.


In [42]:
df_cleaned_5.shape

(56835, 374)

##7. Datatype Inconsistencies

In [43]:
df_cleaned_6 = df_cleaned_5.copy()

In [44]:
df_cleaned_6.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 56835 entries, 0 to 56916
Data columns (total 374 columns):
 #    Column                                           Non-Null Count  Dtype         
---   ------                                           --------------  -----         
 0    subject_id                                       56835 non-null  int64         
 1    hadm_id                                          56835 non-null  int64         
 2    procedures_applied                               30242 non-null  object        
 3    icd_code                                         56835 non-null  object        
 4    long_title                                       56835 non-null  object        
 5    seq_num                                          56835 non-null  int64         
 6    Classification                                   56835 non-null  object        
 7    included                                         56835 non-null  object        
 8    class                        

In [45]:
# Since ML algorithms only take in float and int datatypes, resolving other inconsistent datatypes.
# Checking which columns need to be transformed.
df_cleaned_6.select_dtypes(exclude = ['int64','int32','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Index: 56835 entries, 0 to 56916
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   procedures_applied          30242 non-null  object        
 1   icd_code                    56835 non-null  object        
 2   long_title                  56835 non-null  object        
 3   Classification              56835 non-null  object        
 4   included                    56835 non-null  object        
 5   admittime                   56835 non-null  datetime64[ns]
 6   dischtime                   56835 non-null  datetime64[ns]
 7   edregtime                   39169 non-null  object        
 8   edouttime                   39169 non-null  object        
 9   icd_code_list               56835 non-null  object        
 10  next_admittime              21191 non-null  object        
 11  bmi_chartdate               48902 non-null  object        


In [46]:
#Dropping some features because they have been used as a part of feature extraction process, not required due to datatype mismatch, or data leakage.
# List of columns to drop
cols_to_drop1 = ['procedures_applied','icd_code','edregtime','edouttime','long_title','admittime','dischtime','Classification','included','icd_code_list',
                'next_admittime','bmi_chartdate','careunit_transfer_sequence','first_culture_time','spec_types','first_med_time','first_lab_time',
                 'first_abnormal_lab_time','medications_given','drug_list']

#cols_to_drop1 = ['procedures_applied','edregtime','edouttime','long_title','admittime','dischtime','included','icd_code_list',
#                'bmi_chartdate','careunit_transfer_sequence','first_culture_time','spec_types','first_med_time','first_lab_time','first_abnormal_lab_time',
#                'medications_given','drug_list']

df_cleaned_6.drop(columns=[col for col in cols_to_drop1 if col in df_cleaned_6.columns], inplace=True)

In [47]:
# Checking which columns are left
df_cleaned_6.select_dtypes(exclude = ['int64','int32','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Index: 56835 entries, 0 to 56916
Empty DataFrame


In [48]:
df_cleaned_6.head()

,subject_id,hadm_id,seq_num,class,gender,anchor_age,anchor_year,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,days_to_next_admission,time_to_diagnosis,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,prn_med_count,num_prescriptions_drugs,num_unique_prescriptions_drugs,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_alanine_aminotransferase_(alt),first_value_albumin,first_value_alkaline_phosphatase,first_value_anion_gap,first_value_asparate_aminotransferase_(ast),first_value_basophils,first_value_bicarbonate,"first_value_bilirubin,_total","first_value_calcium,_total",first_value_chloride,first_value_creatinine,first_value_eosinophils,first_value_glucose,first_value_h,first_value_hematocrit,first_value_hemoglobin,first_value_i,first_value_inr(pt),first_value_immature_granulocytes,first_value_l,first_value_lactate,first_value_lymphocytes,first_value_mch,first_value_mchc,first_value_mcv,first_value_magnesium,first_value_monocytes,first_value_neutrophils,first_value_pt,first_value_ptt,first_value_phosphate,first_value_platelet_count,first_value_potassium,first_value_rdw,first_value_rdw-sd,first_value_red_blood_cells,first_value_sodium,first_value_specific_gravity,first_value_urea_nitrogen,first_value_white_blood_cells,first_value_ph,max_value_absolute_basophil_count,max_value_absolute_eosinophil_count,max_value_absolute_lymphocyte_count,max_value_absolute_monocyte_count,max_value_absolute_neutrophil_count,max_value_alanine_aminotransferase_(alt),max_value_albumin,max_value_alkaline_phosphatase,max_value_anion_gap,max_value_asparate_aminotransferase_(ast),max_value_basophils,max_value_bicarbonate,"max_value_bilirubin,_total","max_value_calcium,_total",max_value_chloride,max_value_creatinine,max_value_eosinophils,max_value_glucose,max_value_h,max_value_hematocrit,max_value_hemoglobin,max_value_i,max_value_inr(pt),max_value_immature_granulocytes,max_value_l,max_value_lactate,max_value_lymphocytes,max_value_mch,max_value_mchc,max_value_mcv,max_value_magnesium,max_value_monocytes,max_value_neutrophils,max_value_pt,max_value_ptt,max_value_phosphate,max_value_platelet_count,max_value_potassium,max_value_rdw,max_value_rdw-sd,max_value_red_blood_cells,max_value_sodium,max_value_specific_gravity,max_value_urea_nitrogen,max_value_white_blood_cells,max_value_ph,min_value_absolute_basophil_count,min_value_absolute_eosinophil_count,min_value_absolute_lymphocyte_count,m

In [49]:
df_cleaned_6.shape

(56835, 354)

##8. Range and Distribution Checks

In [50]:
df_cleaned_6.describe()

,subject_id,hadm_id,seq_num,class,gender,anchor_age,anchor_year,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,days_to_next_admission,time_to_diagnosis,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,prn_med_count,num_prescriptions_drugs,num_unique_prescriptions_drugs,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_alanine_aminotransferase_(alt),first_value_albumin,first_value_alkaline_phosphatase,first_value_anion_gap,first_value_asparate_aminotransferase_(ast),first_value_basophils,first_value_bicarbonate,"first_value_bilirubin,_total","first_value_calcium,_total",first_value_chloride,first_value_creatinine,first_value_eosinophils,first_value_glucose,first_value_h,first_value_hematocrit,first_value_hemoglobin,first_value_i,first_value_inr(pt),first_value_immature_granulocytes,first_value_l,first_value_lactate,first_value_lymphocytes,first_value_mch,first_value_mchc,first_value_mcv,first_value_magnesium,first_value_monocytes,first_value_neutrophils,first_value_pt,first_value_ptt,first_value_phosphate,first_value_platelet_count,first_value_potassium,first_value_rdw,first_value_rdw-sd,first_value_red_blood_cells,first_value_sodium,first_value_specific_gravity,first_value_urea_nitrogen,first_value_white_blood_cells,first_value_ph,max_value_absolute_basophil_count,max_value_absolute_eosinophil_count,max_value_absolute_lymphocyte_count,max_value_absolute_monocyte_count,max_value_absolute_neutrophil_count,max_value_alanine_aminotransferase_(alt),max_value_albumin,max_value_alkaline_phosphatase,max_value_anion_gap,max_value_asparate_aminotransferase_(ast),max_value_basophils,max_value_bicarbonate,"max_value_bilirubin,_total","max_value_calcium,_total",max_value_chloride,max_value_creatinine,max_value_eosinophils,max_value_glucose,max_value_h,max_value_hematocrit,max_value_hemoglobin,max_value_i,max_value_inr(pt),max_value_immature_granulocytes,max_value_l,max_value_lactate,max_value_lymphocytes,max_value_mch,max_value_mchc,max_value_mcv,max_value_magnesium,max_value_monocytes,max_value_neutrophils,max_value_pt,max_value_ptt,max_value_phosphate,max_value_platelet_count,max_value_potassium,max_value_rdw,max_value_rdw-sd,max_value_red_blood_cells,max_value_sodium,max_value_specific_gravity,max_value_urea_nitrogen,max_value_white_blood_cells,max_value_ph,min_value_absolute_basophil_count,min_value_absolute_eosinophil_count,min_value_absolute_lymphocyte_count,m

In [51]:
#Getting the statistical summary
summary = df_cleaned_6.describe(include='all').T
summary['missing_count'] = df_cleaned_6.isnull().sum()
summary['missing_percent'] = 100 * summary['missing_count'] / len(df_cleaned_6)

# Add variance
summary['variance'] = df_cleaned_6.var()

# Add flag columns
summary['zero_variance'] = summary['variance'] == 0
summary['has_negatives'] = (df_cleaned_6.select_dtypes(include=[np.number]) < 0).any()
summary['std_too_high'] = summary['std'] > summary['mean'] * 2  # Heuristic
summary['min_out_of_bounds'] = summary['min'] < 0  # Can refine with known clinical bounds
summary['max_outlier_flag'] = summary['max'] > summary['75%'] + 3 * (summary['75%'] - summary['25%'])  # Extreme outliers
summary['high_iqr'] = (summary['75%'] - summary['25%']) > summary['mean']  # Wide spread
summary['low_iqr'] = (summary['75%'] - summary['25%']) < summary['std'] * 0.1  # Flat distribution

# Optional: define known unit or logic violations
suspicious_columns = [
    col for col in df_cleaned_6.columns
    if any(word in col.lower() for word in ['duration', 'hrs', 'age', 'gap', 'rate'])
]
summary['suspicious_negatives'] = summary.index.isin(suspicious_columns) & summary['min_out_of_bounds']

# Final filter: show only flagged rows
flags = summary[
    (summary['missing_percent'] > 0) |
    summary['zero_variance'] |
    summary['has_negatives'] |
    summary['std_too_high'] |
    summary['min_out_of_bounds'] |
    summary['max_outlier_flag'] |
    summary['high_iqr'] |
    summary['low_iqr'] |
    summary['suspicious_negatives']
].copy()

# Reorder columns for clarity
columns_order = [
    'count', 'missing_count', 'missing_percent', 'mean', 'std', 'variance',
    'min', '25%', '50%', '75%', 'max',
    'zero_variance', 'has_negatives', 'std_too_high',
    'min_out_of_bounds', 'max_outlier_flag', 'high_iqr', 'low_iqr', 'suspicious_negatives'
]

flags = flags[columns_order]

# Display the flagged summary
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [52]:
flags

,count,missing_count,missing_percent,mean,std,variance,min,25%,50%,75%,max,zero_variance,has_negatives,std_too_high,min_out_of_bounds,max_outlier_flag,high_iqr,low_iqr,suspicious_negatives
seq_num,56835.000,0,0.000,5.119,4.578,20.956,1.000,2.000,4.000,7.000,39.000,False,False,False,False,True,False,False,False
admission_type,56835.000,0,0.000,2.937,1.944,3.780,1.000,1.000,2.000,4.000,9.000,False,False,False,False,False,True,False,False
admission_location,56835.000,0,0.000,2.243,1.435,2.059,1.000,1.000,2.000,3.000,11.000,False,False,False,False,True,False,False,False
discharge_location,56835.000,0,0.000,4.940,5.157,26.599,1.000,1.000,2.000,7.000,14.000,False,False,False,False,False,True,False,False
language,56835.000,0,0.000,1.105,0.307,0.094,1.000,1.000,1.000,1.000,2.000,False,False,False,False,True,False,True,False
race,56835.000,0,0.000,1.856,1.521,2.315,1.000,1.000,1.000,2.000,6.000,False,False,False,False,True,False,False,False
hospital_expire_flag,56835.000,0,0.000,0.031,0.172,0.030,0.000,0.000,0.000,0.000,1.000,False,False,True,False,True,False,True,False
is_first_admission,56835.000,0,0.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000,True,False,False,False,False,False,False,False
num_followups,56835.000,0,0.000,0.925,2.073,4.297,0.000,0.000,0.000,1.000,50.000,False,False,True,False,True,True,False,False
future_rare_skin,56835.000,0,0.000,0.014,0.118,0.014,0.000,0.000,0.000,0.000,1.000,False,False,True,False,True,False,True,False


Checking for zero variance, none of the features have that. Not removing any outliers because they might be important in rare diseases research.

##9. Missing Data - DON'T RUN DIRECTLY IT HAS CASES

In [53]:
df_cleaned_7 = df_cleaned_6.copy()

In [54]:
def check_missing_info(df, sort=True, show_only_missing=True):
    """
    Returns a DataFrame showing missing counts and percentages for each column.
    """
    missing_info = df.isnull().sum()
    missing_percent = 100 * df.isnull().mean()
    result = pd.DataFrame({
        'missing_count': missing_info,
        'missing_percent': missing_percent
    })
    if show_only_missing:
        result = result[result['missing_count'] > 0]
    if sort:
        result = result.sort_values(by='missing_count', ascending=False)

    print(f"Total features with missing data: {result.shape[0]}")
    return result

In [55]:
check_missing_info(df_cleaned_7)

Total features with missing data: 242


,missing_count,missing_percent
first_value_lactate,39750,69.939
max_value_lactate,39750,69.939
last_value_lactate,39750,69.939
min_value_lactate,39750,69.939
max_value_albumin,37846,66.589
min_value_albumin,37846,66.589
first_value_albumin,37846,66.589
last_value_albumin,37846,66.589
max_value_specific_gravity,37723,66.373
first_value_specific_gravity,37723,66.373


The frequency_value_ columns represent the number of times a specific lab test was conducted during a patient’s hospital stay. Missing values (NaN) in these columns do not indicate data collection errors or unknown values, but rather the absence of that specific lab test for the patient. In clinical terms, if a test wasn’t performed, its frequency is effectively zero. Replacing these NaN values with 0 is therefore semantically correct and clinically meaningful, as it: Reflects the true absence of a test rather than uncertainty. Preserves the interpretability of these features as explicit lab test counts. Avoids introducing bias that could occur with statistical imputation (mean/median).

In [56]:
# Identify columns that start with 'frequency_value_'
freq_cols = [col for col in df_cleaned_7.columns if col.startswith('frequency_value_')]
# Replace NaN with 0 in those columns
df_cleaned_7[freq_cols] = df_cleaned_7[freq_cols].fillna(0)

In [57]:
check_missing_info(df_cleaned_7)

Total features with missing data: 196


,missing_count,missing_percent
first_value_lactate,39750,69.939
last_value_lactate,39750,69.939
min_value_lactate,39750,69.939
max_value_lactate,39750,69.939
min_value_albumin,37846,66.589
first_value_albumin,37846,66.589
max_value_albumin,37846,66.589
last_value_albumin,37846,66.589
max_value_specific_gravity,37723,66.373
min_value_specific_gravity,37723,66.373


In electronic health record (EHR) data, particularly in critical care datasets such as MIMIC-IV, missing values are common and can carry meaningful clinical information. Proper handling of these missing data points is crucial for building robust machine learning models. Our imputation strategy carefully considers the clinical context behind missingness in each feature to avoid bias and maintain data integrity.
1. Time-to-Event Variables:
For features representing time intervals (e.g., time_to_emergency_hrs, time_to_first_abnormal_lab_hrs, time_to_first_lab_hrs, time_to_first_med_hrs), missing values indicate the absence of the corresponding event (e.g., no emergency admission, no abnormal lab result, no lab test, or no medication administered). In these cases, we imputed missing values with 0, reflecting that the event did not occur during the patient’s observation period. This approach logically distinguishes between “no event” and “unknown time,” allowing the model to leverage the absence of the event as an informative feature rather than discarding or arbitrarily imputing data.
2. Medication-Related Counts and Dosages:
Features related to medication administration, such as prn_med_count, avg_dose_val, max_dose_val, and medication duration variables, were similarly imputed with 0 when missing. A missing value here implies that no medication was given, so setting these to zero accurately reflects the absence of medication exposure, preventing misleading assumptions about dosage or duration.
3. Body Mass Index (BMI):
Unlike the time or medication features, missing BMI values are typically due to not being recorded rather than the clinical absence of a condition. BMI is a continuous clinical measurement important for modeling patient health status. Therefore, we imputed missing BMI values using the median BMI calculated from the available data to preserve the distribution and minimize skewness. When appropriate, a more refined imputation was applied by computing median BMI stratified by patient demographic groups such as age, race and gender, capturing relevant physiological variation.

In [58]:
cols_to_impute_zero = ['time_to_emergency_hrs','time_to_first_abnormal_lab_hrs','time_to_first_lab_hrs','time_to_first_med_hrs','prn_med_count',
                       'avg_dose_val','max_dose_val','avg_duration_days','max_duration_days','avg_med_duration_hrs','max_med_duration_hrs']

df_cleaned_7[cols_to_impute_zero] = df_cleaned_7[cols_to_impute_zero].fillna(0)

#Fill missing BMI with groupwise median first
df_cleaned_7['bmi'] = df_cleaned_7.groupby(['gender', 'race'])['bmi'] \
                                   .transform(lambda x: x.fillna(x.median()))
# Then fill any remaining missing BMI with overall median
df_cleaned_7['bmi'] = df_cleaned_7['bmi'].fillna(df_cleaned_7['bmi'].median())

In [59]:
check_missing_info(df_cleaned_7)

Total features with missing data: 184


,missing_count,missing_percent
first_value_lactate,39750,69.939
last_value_lactate,39750,69.939
min_value_lactate,39750,69.939
max_value_lactate,39750,69.939
min_value_albumin,37846,66.589
first_value_albumin,37846,66.589
max_value_albumin,37846,66.589
last_value_albumin,37846,66.589
max_value_specific_gravity,37723,66.373
min_value_specific_gravity,37723,66.373


The dataset includes multiple versions of the same laboratory measurements, such as first_value, min_value, max_value, and last_value. Among these, we chose to drop the last_value_* features for the following reasons:
1. last_value_* columns represent the most recent lab results during a patient's stay, which are often only available after the disease course has progressed or near discharge. Including these can lead to data leakage, where the model has access to information that wouldn’t be available in real-time clinical settings.
2. In critically ill patients or those with short hospital stays, last_value_* may not be representative or even available. This contributes to higher missingness and decreases the generalizability of the model.
3. first_value_*, min_value_*, and max_value_* provide clinically relevant information about the baseline state and severity of abnormalities during the admission. These features are more informative for early prediction models and align better with real-world clinical workflows.
4. By removing last_value_* features, we ensure the model is trained on data that would realistically be available at the time of prediction, thus preserving temporal validity and supporting prospective deployment of the model in clinical settings.



In [60]:
# Identify columns that start with 'last_value_'
last_value_cols = [col for col in df_cleaned_7.columns if col.startswith('last_value_')]
# Drop those columns
df_cleaned_7 = df_cleaned_7.drop(columns=last_value_cols)

Checking the class of all these missing records, if it is the classes we have enough data for, those records can simply be dropped.

In [61]:
missing_cols = check_missing_info(df_cleaned_7)
lab_cols = missing_cols.index.tolist()
# Missing counts per class
missing_by_class = df_cleaned_7.groupby('class')[lab_cols].apply(lambda x: x.isnull().sum())
# Missing percentages per class
missing_by_class_percent = df_cleaned_7.groupby('class')[lab_cols].apply(lambda x: x.isnull().mean() * 100)

Total features with missing data: 138


In [62]:
df_cleaned_7['class'].value_counts()

,count
class,
4,28998
3,21951
2,2274
5,2198
1,1414


In [63]:
missing_by_class

,max_value_lactate,first_value_lactate,min_value_lactate,max_value_albumin,first_value_albumin,min_value_albumin,first_value_specific_gravity,max_value_specific_gravity,min_value_specific_gravity,max_value_immature_granulocytes,min_value_immature_granulocytes,first_value_immature_granulocytes,max_value_neutrophils,min_value_neutrophils,max_value_absolute_basophil_count,max_value_absolute_monocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_absolute_basophil_count,min_value_absolute_basophil_count,min_value_absolute_neutrophil_count,min_value_absolute_monocyte_count,first_value_neutrophils,max_value_absolute_neutrophil_count,max_value_absolute_eosinophil_count,first_value_absolute_eosinophil_count,min_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,min_value_absolute_lymphocyte_count,max_value_absolute_lymphocyte_count,max_value_basophils,first_value_basophils,min_value_basophils,min_value_eosinophils,first_value_eosinophils,max_value_eosinophils,first_value_monocytes,min_value_monocytes,max_value_monocytes,max_value_lymphocytes,first_value_lymphocytes,min_value_lymphocytes,min_value_ph,max_value_ph,first_value_ph,"max_value_bilirubin,_total","min_value_bilirubin,_total","first_value_bilirubin,_total",max_value_alkaline_phosphatase,first_value_alkaline_phosphatase,min_value_alkaline_phosphatase,min_value_alanine_aminotransferase_(alt),first_value_alanine_aminotransferase_(alt),max_value_alanine_aminotransferase_(alt),max_value_asparate_aminotransferase_(ast),first_value_asparate_aminotransferase_(ast),min_value_asparate_aminotransferase_(ast),min_value_ptt,max_value_ptt,first_value_ptt,min_value_inr(pt),max_value_inr(pt),first_value_inr(pt),first_value_pt,min_value_pt,max_value_pt,first_value_i,max_value_i,min_value_i,min_value_l,max_value_l,max_value_h,first_value_h,first_value_l,min_value_h,first_value_phosphate,min_value_phosphate,max_value_phosphate,"max_value_calcium,_total","min_value_calcium,_total","first_value_calcium,_total",max_value_magnesium,min_value_magnesium,first_value_magnesium,min_value_glucose,max_value_glucose,first_value_glucose,first_value_anion_gap,max_value_anion_gap,min_value_anion_gap,first_value_bicarbonate,max_value_bicarbonate,min_value_bicarbonate,first_value_chloride,max_value_chloride,min_value_chloride,min_value_sodium,first_value_sodium,max_value_sodium,first_value_urea_nitrogen,max_value_urea_nitrogen,min_value_urea_nitrogen,min_value_rdw-sd,max_value_rdw-sd,first_value_rdw-sd,first_value_rdw,max_value_rdw,min_value_rdw,min_value_mch,max_value_mchc,max_value_mch,first_value_mch,min_value_mchc,first_value_mchc,max_value_mcv,min_value_mcv,first_value_mcv,first_value_red_blood_cells,max_value_red_blood_cells,min_value_red_blood_cells,first_value_potassium,max_value_potassium,min_value_potassium,max_value_white_blood_cells,min_value_white_blood_cells,first_value_white_blood_cells,max_value_hemoglobin,first_value_hemoglobin,min_value_hemoglobin,first_value_creatinine,min_value_creatinine,max_value_creatinine,first_value_platelet_count,max_value_platelet_count,min_value_platelet_count,first_value_hematocrit,max_value_hematocrit,min_value_hematocrit
class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,911,911,911,711,711,711,740,740,740,739,739,739,600,600,600,600,600,600,600,600,600,600,600,600,600,600,600,599,599,599,599,599,599,596,596,596,590,590,590,589,589,589,647,647,647,547,547,547,539,539,539,528,528,528,522,522,522,426,426,426,406,406,406,406,406,406,392,392,392,392,392,392,392,392,392,179,179,179,172,172,172,168,168,168,141,141,141,140,140,140,140,140,140,140,140,140,139,139,139,139,139,139,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,139,139,139,123,123,123,120,120,120,135,135,135,122,122,122,113,113,113
2,1581,1581,1581,1391,1391,1391,1466,1466,1466,1332,1332,1332,1192,1192,1192,1192,1192,1192,1192,1

In [64]:
## There is different CODE for different CASES

# CODE 1
#This gives all rows without missing data in class 3 and 4
# Create a mask for rows in class 3 or 4
#mask_class_3_4 = df_cleaned_7['class'].isin([3, 4])
# Create another mask to identify rows with any missing values
#mask_missing = df_cleaned_7.isnull().any(axis=1)
# Combine both conditions: rows from class 3 or 4 AND having missing values
#rows_to_remove = mask_class_3_4 & mask_missing
# Remove those rows from the DataFrame
#df_cleaned_8 = df_cleaned_7[~rows_to_remove]
# Count the number of rows left
#remaining_rows = df_cleaned_8.shape[0]
#print(f"Total rows remaining after removing rows with missing data from class 3 and 4: {remaining_rows}")


# CODE 2
#This gives rows with missing data in all the classes including class 3 and 4 (all rows with no missing data + 600 rows with missing data)
# Mask for class 3 or 4
mask_class_3_4 = df_cleaned_7['class'].isin([3, 4])
# Mask for rows with missing data
mask_missing = df_cleaned_7.isnull().any(axis=1)
# Mask for future_rare_skin == 1
mask_future_rare_skin = df_cleaned_7['future_rare_skin'] == 1
# Class 3/4 rows WITH missing values (excluding future_rare_skin == 1, which will be handled separately)
class_3_4_missing = df_cleaned_7[mask_class_3_4 & mask_missing & ~mask_future_rare_skin]
# Class 3/4 rows WITHOUT missing values
class_3_4_no_missing = df_cleaned_7[mask_class_3_4 & ~mask_missing]
# Sample 600 from the class 3/4 rows that have missing data (excluding future_rare_skin == 1)
class_3_4_missing_sampled = class_3_4_missing.sample(n=min(500, len(class_3_4_missing)), random_state=42)
# Always keep all rows with future_rare_skin == 1 (regardless of class or missingness)
future_rare_skin_rows = df_cleaned_7[mask_future_rare_skin]
# Keep all rows that are NOT class 3/4
not_class_3_4 = df_cleaned_7[~mask_class_3_4]
# Combine everything together:
df_cleaned_8 = pd.concat([not_class_3_4,class_3_4_no_missing,class_3_4_missing_sampled,future_rare_skin_rows], ignore_index=True).drop_duplicates()
# Verification
total_class_3_4_missing = df_cleaned_8[df_cleaned_8['class'].isin([3, 4])].isnull().any(axis=1).sum()
total_class_3_4 = df_cleaned_8['class'].isin([3, 4]).sum()
future_rare_retained = df_cleaned_8['future_rare_skin'].sum()
print(f"Retained class 3/4 records with missing data: {total_class_3_4_missing}")
print(f"Total class 3/4 records: {total_class_3_4}")
print(f"Total rows in final DataFrame: {df_cleaned_8.shape[0]}")
print(f"Total rows with future_rare_skin == 1: {future_rare_retained}")


# CODE 3
#This gives all rows without missing data in class 1, 2, 3, 4 and 5
# Create a mask for rows in all classes
#mask_class_all = df_cleaned_7['class'].isin([1, 2, 3, 4, 5])
# Create another mask to identify rows with any missing values
#mask_missing = df_cleaned_7.isnull().any(axis=1)
# Combine both conditions: rows from class 3 or 4 AND having missing values
#rows_to_remove = mask_class_all & mask_missing
# Remove those rows from the DataFrame
#df_cleaned_8 = df_cleaned_7[~rows_to_remove]
# Count the number of rows left
#remaining_rows = df_cleaned_8.shape[0]
#print(f"Total rows remaining after removing rows with missing data from all classes: {remaining_rows}")

Retained class 3/4 records with missing data: 1175
Total class 3/4 records: 4953
Total rows in final DataFrame: 10839
Total rows with future_rare_skin == 1: 806


In [65]:
df_cleaned_8['class'].value_counts()

,count
class,
3,2487
4,2466
2,2274
5,2198
1,1414


In clinical datasets such as MIMIC-IV, missing values are often not missing at random but rather indicative of clinical decisions or patient pathways (e.g., a lab test not ordered because it was deemed unnecessary). Instead of imputing these missing lab values (e.g., first_value_, min_value_, max_value_) with statistical estimates such as the mean or median, which could dilute meaningful absence we employed a sentinel value strategy using -99999 to explicitly encode missingness.
1. Preserves Signal of Missingness: In healthcare data, missingness can be informative. For instance, certain lab tests may only be ordered for high-risk patients. Encoding with -99999 allows tree-based models to learn splits based on the presence or absence of values without treating them as noise.
2. Avoids Distributional Bias: Unlike mean or median imputation, which can introduce artificial clustering, sentinel values maintain the integrity of the observed distribution by clearly separating imputed values.
3. Optimized for Tree-Based Models: Models such as Random Forest, XGBoost, LightGBM, AdaBoost, and ExtraTrees are robust to outliers and do not require feature scaling or normalization. These models handle extreme values gracefully and can isolate -99999 as a split point during training, effectively treating it as a distinct category.
4. Avoids Leakage and Simplifies Preprocessing: Using a fixed sentinel eliminates the risk of data leakage from using population-based statistics (e.g., global medians) and simplifies pipeline design, particularly when data is split into train-test folds.


In [66]:
# Select columns with these prefixes
target_cols = [col for col in df_cleaned_8.columns if col.startswith(('first_value_', 'min_value_', 'max_value_'))]
# Replace NaNs in those columns with -99999
df_cleaned_8[target_cols] = df_cleaned_8[target_cols].fillna(-99999)

In [67]:
check_missing_info(df_cleaned_8)

Total features with missing data: 0


,missing_count,missing_percent


In [68]:
df_cleaned_8.shape

(10839, 308)

##10. Highly Correlated Features - DON'T RUN DIRECTLY IT HAS CASES

In [69]:
df_cleaned_9 = df_cleaned_8.copy()

In [70]:
#Compute the correlation matrix
corr_matrix = df_cleaned_9.corr(numeric_only=True)
#Create a boolean mask of features with high correlation (> 0.9)
high_corr_mask = (abs(corr_matrix) > 0.9) & (abs(corr_matrix) < 1.0)
#Get the columns involved in high correlation
high_corr_cols = high_corr_mask.any(axis=1)
filtered_corr_matrix = corr_matrix.loc[high_corr_cols, high_corr_cols]
#Plot the heatmap for only highly correlated features
#plt.figure(figsize=(20, 20))
#sns.heatmap(filtered_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
#plt.title("Highly Correlated Features (|correlation| > 0.9)")
#plt.tight_layout()
#plt.show()

In [71]:
# Get the upper triangle of the correlation matrix to avoid duplicates
upper_tri = filtered_corr_matrix.where(np.triu(np.ones(filtered_corr_matrix.shape), k=1).astype(bool))

# Find row/column pairs with correlation > 0.9
high_corr_pairs = upper_tri.stack().reset_index()
high_corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
high_corr_pairs = high_corr_pairs[abs(high_corr_pairs['Correlation']) > 0.9]

# Display the pairs
print(high_corr_pairs.sort_values(by='Correlation', ascending=False))

                                             Feature 1                                        Feature 2  Correlation
21901                              frequency_value_rdw                  frequency_value_red_blood_cells        1.000
21784                              frequency_value_mcv                  frequency_value_red_blood_cells        1.000
21782                              frequency_value_mcv                              frequency_value_rdw        1.000
21766                             frequency_value_mchc                  frequency_value_red_blood_cells        1.000
21755                             frequency_value_mchc                              frequency_value_mcv        1.000
21764                             frequency_value_mchc                              frequency_value_rdw        1.000
21736                              frequency_value_mch                              frequency_value_mcv        1.000
21735                              frequency_value_mch          

Why drop min_value_* and max_value_* columns?
High Redundancy: In many lab tests, min and max are highly correlated (often > 0.90), especially when:
Tests were performed only once.
Values didn’t fluctuate significantly over time.
Avoid Multicollinearity:
For interpretable models or non-tree-based methods, such as MLPs or logistic regression, high multicollinearity harms performance.
Even for tree-based models, keeping redundant features adds computational overhead and noise.
First Values Are Clinically Meaningful:
First lab values often reflect initial patient condition at admission, making them more useful for risk stratification and early prediction.
Many clinical studies prioritize baseline/first values for modeling outcomes.

https://arxiv.org/pdf/1703.07771

https://pmc.ncbi.nlm.nih.gov/articles/PMC5971671/

https://pmc.ncbi.nlm.nih.gov/articles/PMC5228148/


In [72]:
# Identify and drop all columns starting with 'min_value_' or 'max_value_'
cols_to_drop = [col for col in df_cleaned_9.columns if col.startswith('min_value_') or col.startswith('max_value_')]
# Drop them from the dataframe
df_cleaned_9 = df_cleaned_9.drop(columns=cols_to_drop)
print(f"Dropped {len(cols_to_drop)} columns: min_value_*, max_value_*")

Dropped 92 columns: min_value_*, max_value_*


In [73]:
# 1. Compute the correlation matrix (numeric only)
corr_matrix = df_cleaned_9.corr(numeric_only=True)

# 2. Create a boolean mask for |correlation| > 0.90 and < 1.0 (exclude self-correlation)
high_corr_mask = (abs(corr_matrix) > 0.90) & (abs(corr_matrix) < 1.0)

# 3. Extract only the pairs of features that satisfy the condition
high_corr_pairs = high_corr_mask.stack().loc[lambda x: x].reset_index()
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# 4. Add actual correlation values
high_corr_pairs['Correlation'] = high_corr_pairs.apply(lambda row: corr_matrix.loc[row['Feature_1'], row['Feature_2']], axis=1)

# 5. Drop duplicate pairs (e.g., A-B and B-A)
high_corr_pairs_sorted = high_corr_pairs.apply(lambda row: sorted([row['Feature_1'], row['Feature_2']]), axis=1, result_type='expand')
high_corr_pairs['sorted_pair'] = high_corr_pairs_sorted[0] + "_" + high_corr_pairs_sorted[1]
high_corr_pairs = high_corr_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

# 6. Display the results
print("Highly correlated feature pairs (|correlation| > 0.90):")
display(high_corr_pairs.sort_values(by='Correlation', ascending=False))

Highly correlated feature pairs (|correlation| > 0.90):


,Feature_1,Feature_2,Correlation
89,max_duration_days,max_med_duration_hrs,1.000
148,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,1.000
150,first_value_absolute_basophil_count,first_value_absolute_monocyte_count,1.000
160,first_value_absolute_eosinophil_count,first_value_absolute_monocyte_count,1.000
329,first_value_mchc,first_value_red_blood_cells,1.000
315,first_value_mch,first_value_mchc,1.000
320,first_value_mch,first_value_red_blood_cells,1.000
151,first_value_absolute_basophil_count,first_value_absolute_neutrophil_count,1.000
181,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,1.000
161,first_value_absolute_eosinophil_count,first_value_absolute_neutrophil_count,1.000


In [74]:
# Filter out pairs where either column starts with "first_value_" or "frequency_value_"
filtered_high_corr_pairs = high_corr_pairs[
    ~(
        high_corr_pairs['Feature_1'].str.startswith(('first_value_', 'frequency_value_')) |
        high_corr_pairs['Feature_2'].str.startswith(('first_value_', 'frequency_value_'))
    )
]

# Display the remaining correlated feature pairs
print("Highly correlated feature pairs (excluding 'first_value_' and 'frequency_value_'):")
display(filtered_high_corr_pairs.sort_values(by='Correlation', ascending=False))

Highly correlated feature pairs (excluding 'first_value_' and 'frequency_value_'):


,Feature_1,Feature_2,Correlation
89,max_duration_days,max_med_duration_hrs,1.000
46,route_oral,oral_prescribed_route,1.000
48,route_skin,skin_prescribed_route,0.999
44,route_iv,iv_prescribed_route,0.997
88,avg_duration_days,avg_med_duration_hrs,0.995
59,unique_medications,num_unique_prescriptions_drugs,0.995
53,total_medications,num_prescriptions_drugs,0.991
92,total_lab_tests,abnormal_lab_count,0.983
1,total_no_of_procedures,no_of_unique_procedures,0.971
43,route_iv,num_prescriptions_drugs,0.968


In [75]:
# List of highly correlated columns to drop
high_corr_to_drop = [
    'max_med_duration_hrs',
    'oral_prescribed_route',
    'skin_prescribed_route',
    'iv_prescribed_route',
    'avg_med_duration_hrs',
    'num_unique_prescriptions_drugs',
    'num_prescriptions_drugs',
    'total_lab_tests',
    'total_no_of_procedures',
    'count_medications',
    'susceptible_tests',
    'count_lab',
    'total_medications',
    'others_prescribed_route',
    'total_med_duration_hrs',
    'negative_cultures'
]

# Drop the columns from df_cleaned_9
df_cleaned_9.drop(columns=high_corr_to_drop, inplace=True)

Checking highly correlated features with the target is class which will be removed post dataset split.

In [76]:
# Compute correlation matrix with target
corr_with_target = df_cleaned_9.corr(numeric_only=True)['target'].drop('target')

# Filter for features with high absolute correlation (e.g., > 0.2)
high_corr_with_target = corr_with_target[abs(corr_with_target) > 0.2].sort_values(ascending=False)

# Display the results
print("Features highly correlated with target:")
display(high_corr_with_target)

Features highly correlated with target:


,target
class,-0.634


In [77]:
df_cleaned_9.shape

(10839, 200)

In [78]:
df_cleaned_9.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10839 entries, 0 to 10969
Data columns (total 200 columns):
 #    Column                                           Non-Null Count  Dtype  
---   ------                                           --------------  -----  
 0    subject_id                                       10839 non-null  int64  
 1    hadm_id                                          10839 non-null  int64  
 2    seq_num                                          10839 non-null  int64  
 3    class                                            10839 non-null  int64  
 4    gender                                           10839 non-null  int64  
 5    anchor_age                                       10839 non-null  int64  
 6    anchor_year                                      10839 non-null  int64  
 7    admission_type                                   10839 non-null  int64  
 8    admission_location                               10839 non-null  int64  
 9    discharge_location  

In [79]:
#Checking the columns left which are highly correlated.
# 1. Compute the correlation matrix (numeric only)
corr_matrix = df_cleaned_9.corr(numeric_only=True)

# 2. Create a boolean mask for |correlation| > 0.90 and < 1.0 (exclude self-correlation)
high_corr_mask = (abs(corr_matrix) > 0.90) & (abs(corr_matrix) < 1.0)

# 3. Extract only the pairs of features that satisfy the condition
high_corr_pairs = high_corr_mask.stack().loc[lambda x: x].reset_index()
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# 4. Add actual correlation values
high_corr_pairs['Correlation'] = high_corr_pairs.apply(lambda row: corr_matrix.loc[row['Feature_1'], row['Feature_2']], axis=1)

# 5. Drop duplicate pairs (e.g., A-B and B-A)
high_corr_pairs_sorted = high_corr_pairs.apply(lambda row: sorted([row['Feature_1'], row['Feature_2']]), axis=1, result_type='expand')
high_corr_pairs['sorted_pair'] = high_corr_pairs_sorted[0] + "_" + high_corr_pairs_sorted[1]
high_corr_pairs = high_corr_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

# 6. Display the results
print("Highly correlated feature pairs (|correlation| > 0.90):")
display(high_corr_pairs.sort_values(by='Correlation', ascending=False))

Highly correlated feature pairs (|correlation| > 0.90):


,Feature_1,Feature_2,Correlation
26,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,1.000
28,first_value_absolute_basophil_count,first_value_absolute_monocyte_count,1.000
38,first_value_absolute_eosinophil_count,first_value_absolute_monocyte_count,1.000
207,first_value_mchc,first_value_red_blood_cells,1.000
193,first_value_mch,first_value_mchc,1.000
198,first_value_mch,first_value_red_blood_cells,1.000
29,first_value_absolute_basophil_count,first_value_absolute_neutrophil_count,1.000
59,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,1.000
39,first_value_absolute_eosinophil_count,first_value_absolute_neutrophil_count,1.000
194,first_value_mch,first_value_mcv,1.000


**Feature Reduction Justification: Dropping Highly Correlated Variables**

In this study, we performed feature reduction to improve the performance, interpretability, and generalizability of our machine learning models developed for the classification of rare skin diseases using MIMIC-IV data. After identifying 68 highly correlated feature pairs (Pearson’s |r| > 0.90), we retained one variable from each pair based on **clinical relevance**, **information content**, and **redundancy**, guided by domain expertise and biomedical best practices.

The following principles informed our decisions:
### **1. Redundant Hematologic Indices**

We removed several red blood cell (RBC) indices that are inherently correlated due to being derived from the same base measurements (e.g., hemoglobin, hematocrit, and RBC count). For example:

* **Mean corpuscular hemoglobin (MCH)** and **mean corpuscular hemoglobin concentration (MCHC)** are calculated from **RBC count and hemoglobin concentration**, and correlate strongly with **mean corpuscular volume (MCV)** and **red cell distribution width (RDW)**.
* In cases of high multicollinearity, we retained **RDW-SD** as it captures variation in red cell size and may reflect systemic inflammation or nutritional deficiencies, which are relevant in certain rare skin conditions (e.g., lupus erythematosus, vasculitis) \[1].

**Dropped**:
`first_value_mch`, `first_value_mchc`, `first_value_mcv`, `first_value_rdw`,
`frequency_value_mch`, `frequency_value_mchc`, `frequency_value_mcv`, `frequency_value_rdw`

---

### **2. Absolute Counts Preferred Over Percentages in Leukocyte Subsets**

Skin diseases such as atopic dermatitis, urticaria, and cutaneous autoimmune disorders often involve immune dysregulation reflected in white blood cell differentials.

* **Absolute counts** of eosinophils, neutrophils, basophils, and monocytes are more clinically informative than relative percentages, especially in the context of inflammation or immunosuppression \[2].
* Absolute counts are also less influenced by fluctuations in total white blood cell count, improving robustness across patients.

**Dropped**:
`first_value_basophils`, `first_value_neutrophils`,
`frequency_value_basophils`, `frequency_value_neutrophils`

---

### **3. Prefer Standardized or Aggregated Metrics**

When two highly correlated metrics represented the same biological process, we favored the more **standardized** or **interpretable** one.

* **INR (International Normalized Ratio)** is preferred over **PT (Prothrombin Time)**, as INR corrects for lab-to-lab variability and is clinically standard for evaluating coagulopathy \[3].
* Between **anion gap** and **bicarbonate**, we retained **bicarbonate** as a direct marker of acid-base status, more interpretable in metabolic contexts.

**Dropped**:
`first_value_pt`, `first_value_anion_gap`, `frequency_value_anion_gap`

---

### **4. Creatinine Preferred Over Urea Nitrogen**

In assessing renal involvement, which is relevant in rare syndromic dermatoses (e.g., lupus nephritis with cutaneous manifestations), **serum creatinine** is a more direct marker of glomerular filtration than **blood urea nitrogen (BUN)**.

* BUN is affected by dietary protein and catabolism, reducing its specificity.
* Creatinine-based assessments are more consistent in machine learning models of chronic disease \[4].

**Dropped**:
`frequency_value_urea_nitrogen`

---

### **5. Duplicate Temporal Representations**

When both `first_value_` and `frequency_value_` variants of the same lab were present, we retained **one representation** to avoid redundancy.

* In this case, **first value** was favored for acutely presenting parameters (e.g., red blood cell count), while **frequency-based** versions were retained only when they added longitudinal insight.

**Dropped**:
`frequency_value_red_blood_cells`

 **References**

\[1] Lee GR. Red cell indices in anemia: interpretation and clinical use. *Am Fam Physician.* 2001;63(6):1239-1246.

\[2] Boguniewicz M, Leung DYM. Atopic dermatitis: a disease of altered skin barrier and immune dysregulation. *Immunol Rev.* 2011;242(1):233-246.

\[3] Hirsh J, et al. Oral anticoagulants: mechanism of action, clinical effectiveness, and optimal therapeutic range. *Chest.* 2001;119(1\_suppl):8S–21S.

\[4] Stevens LA, Levey AS. Measurement of kidney function. *Med Clin North Am.* 2005;89(3):457-473.

In [80]:
features_to_drop1 = [
    'first_value_mch',
    'first_value_mchc',
    'first_value_mcv',
    'first_value_rdw',
    'frequency_value_mch',
    'frequency_value_mchc',
    'frequency_value_mcv',
    'frequency_value_rdw',
    'first_value_basophils',
    'first_value_neutrophils',
    'frequency_value_basophils',
    'frequency_value_neutrophils',
    'frequency_value_absolute_lymphocyte_count',
    'first_value_pt',
    'first_value_anion_gap',
    'frequency_value_anion_gap',
    'frequency_value_urea_nitrogen',
    'frequency_value_red_blood_cells'
]

# Drop the columns from df_cleaned_9
df_cleaned_9.drop(columns=features_to_drop1, inplace=True)

In [81]:
df_cleaned_9.shape

(10839, 182)

In [82]:
#Checking the columns left which are highly correlated.
# 1. Compute the correlation matrix (numeric only)
corr_matrix = df_cleaned_9.corr(numeric_only=True)

# 2. Create a boolean mask for |correlation| > 0.90 and < 1.0 (exclude self-correlation)
high_corr_mask = (abs(corr_matrix) > 0.90) & (abs(corr_matrix) < 1.0)

# 3. Extract only the pairs of features that satisfy the condition
high_corr_pairs = high_corr_mask.stack().loc[lambda x: x].reset_index()
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# 4. Add actual correlation values
high_corr_pairs['Correlation'] = high_corr_pairs.apply(lambda row: corr_matrix.loc[row['Feature_1'], row['Feature_2']], axis=1)

# 5. Drop duplicate pairs (e.g., A-B and B-A)
high_corr_pairs_sorted = high_corr_pairs.apply(lambda row: sorted([row['Feature_1'], row['Feature_2']]), axis=1, result_type='expand')
high_corr_pairs['sorted_pair'] = high_corr_pairs_sorted[0] + "_" + high_corr_pairs_sorted[1]
high_corr_pairs = high_corr_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

# 6. Display the results
print("Highly correlated feature pairs (|correlation| > 0.90):")
display(high_corr_pairs.sort_values(by='Correlation', ascending=False))

Highly correlated feature pairs (|correlation| > 0.90):


,Feature_1,Feature_2,Correlation
19,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,1.000
21,first_value_absolute_basophil_count,first_value_absolute_monocyte_count,1.000
29,first_value_absolute_eosinophil_count,first_value_absolute_monocyte_count,1.000
22,first_value_absolute_basophil_count,first_value_absolute_neutrophil_count,1.000
46,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,1.000
30,first_value_absolute_eosinophil_count,first_value_absolute_neutrophil_count,1.000
118,first_value_i,first_value_l,1.000
323,frequency_value_h,frequency_value_l,1.000
322,frequency_value_h,frequency_value_i,1.000
378,frequency_value_i,frequency_value_l,1.000


🧠 Rationale for Dropping These Features
Redundancy due to high correlation: Many of these lab frequency features are highly correlated because they reflect the ordering behavior of clinicians rather than physiological differences. For example, the number of times eosinophils and basophils are ordered tends to be similar for certain diagnostic routines, but this does not reflect distinct clinical value.

Preference for first-value features: As you noted, the first measured lab value often contains richer clinical information — e.g., signs of inflammation, infection, or organ dysfunction — whereas frequency only captures utilization intensity.

Preserving interpretability and signal quality: Retaining first values allows better interpretability and medical relevance. For example, a high first ALT value is more clinically meaningful than knowing ALT was ordered three times.

Avoiding multicollinearity: Highly correlated variables can distort model training, especially for linear models or when using SHAP values for interpretability. Removing the less clinically meaningful of the pair (frequency) helps maintain robustness.

🧾 Citations Supporting This Approach:
Kamat et al. (2020), "Data Cleaning for EHR-based Machine Learning: Impacts of Outliers and Correlation": Emphasize removal of highly correlated administrative features when a clinical counterpart is available.

Zhao et al. (2021), "Clinical Feature Selection for Risk Prediction in EHRs": Recommends favoring physiological measurements (like first lab values) over test metadata (like frequency).

Rajkomar et al. (2018), "Scalable and accurate deep learning with EHR data": Highlights signal dilution when including repeated measurement metadata (like lab frequency) unnecessarily.



In [83]:
# List of frequency_value_<labs> columns to drop
features_to_drop2 = [
    'frequency_value_absolute_basophil_count',
    'frequency_value_absolute_eosinophil_count',
    'frequency_value_absolute_monocyte_count',
    'frequency_value_absolute_neutrophil_count',
    'frequency_value_eosinophils',
    'frequency_value_lymphocytes',
    'frequency_value_monocytes',
    'frequency_value_white_blood_cells',
    'frequency_value_platelet_count',
    'frequency_value_rdw-sd',
    'frequency_value_alanine_aminotransferase_(alt)',
    'frequency_value_asparate_aminotransferase_(ast)',
    'frequency_value_alkaline_phosphatase',
    'frequency_value_bilirubin,_total',
    'frequency_value_bicarbonate',
    'frequency_value_chloride',
    'frequency_value_potassium',
    'frequency_value_sodium',
    'frequency_value_creatinine',
    'frequency_value_phosphate',
    'frequency_value_calcium,_total',
    'frequency_value_magnesium',
    'frequency_value_hematocrit',
    'frequency_value_hemoglobin'
]

df_cleaned_9.drop(columns=features_to_drop2, inplace=True)

In [84]:
#Checking the columns left which are highly correlated.
# 1. Compute the correlation matrix (numeric only)
corr_matrix = df_cleaned_9.corr(numeric_only=True)

# 2. Create a boolean mask for |correlation| > 0.90 and < 1.0 (exclude self-correlation)
high_corr_mask = (abs(corr_matrix) > 0.90) & (abs(corr_matrix) < 1.0)

# 3. Extract only the pairs of features that satisfy the condition
high_corr_pairs = high_corr_mask.stack().loc[lambda x: x].reset_index()
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# 4. Add actual correlation values
high_corr_pairs['Correlation'] = high_corr_pairs.apply(lambda row: corr_matrix.loc[row['Feature_1'], row['Feature_2']], axis=1)

# 5. Drop duplicate pairs (e.g., A-B and B-A)
high_corr_pairs_sorted = high_corr_pairs.apply(lambda row: sorted([row['Feature_1'], row['Feature_2']]), axis=1, result_type='expand')
high_corr_pairs['sorted_pair'] = high_corr_pairs_sorted[0] + "_" + high_corr_pairs_sorted[1]
high_corr_pairs = high_corr_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

# 6. Display the results
print("Highly correlated feature pairs (|correlation| > 0.90):")
display(high_corr_pairs.sort_values(by='Correlation', ascending=False))

Highly correlated feature pairs (|correlation| > 0.90):


,Feature_1,Feature_2,Correlation
6,first_value_absolute_basophil_count,first_value_absolute_eosinophil_count,1.000
8,first_value_absolute_basophil_count,first_value_absolute_monocyte_count,1.000
16,first_value_absolute_eosinophil_count,first_value_absolute_monocyte_count,1.000
9,first_value_absolute_basophil_count,first_value_absolute_neutrophil_count,1.000
33,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,1.000
17,first_value_absolute_eosinophil_count,first_value_absolute_neutrophil_count,1.000
105,first_value_i,first_value_l,1.000
183,frequency_value_h,frequency_value_l,1.000
182,frequency_value_h,frequency_value_i,1.000
187,frequency_value_i,frequency_value_l,1.000


Check reason in the excel sheet

In [85]:
# List of highly correlated lab features to drop
features_to_drop3 = [
    'first_value_absolute_basophil_count',
    'first_value_absolute_eosinophil_count',
    'first_value_absolute_monocyte_count',
    'first_value_absolute_neutrophil_count',
    'first_value_i',
    'frequency_value_h',
    'frequency_value_i',
    'first_value_h',
    'first_value_rdw-sd',
    'first_value_red_blood_cells',
    'first_value_chloride',
    'first_value_eosinophils',
    'first_value_bicarbonate',
    'first_value_potassium',
    'first_value_hemoglobin',
    'first_value_platelet_count',
    'first_value_monocytes',
]

df_cleaned_9.drop(columns=features_to_drop3, inplace=True)

In [86]:
#Checking the columns left which are highly correlated.
# 1. Compute the correlation matrix (numeric only)
corr_matrix = df_cleaned_9.corr(numeric_only=True)

# 2. Create a boolean mask for |correlation| > 0.90 and < 1.0 (exclude self-correlation)
high_corr_mask = (abs(corr_matrix) > 0.90) & (abs(corr_matrix) < 1.0)

# 3. Extract only the pairs of features that satisfy the condition
high_corr_pairs = high_corr_mask.stack().loc[lambda x: x].reset_index()
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# 4. Add actual correlation values
high_corr_pairs['Correlation'] = high_corr_pairs.apply(lambda row: corr_matrix.loc[row['Feature_1'], row['Feature_2']], axis=1)

# 5. Drop duplicate pairs (e.g., A-B and B-A)
high_corr_pairs_sorted = high_corr_pairs.apply(lambda row: sorted([row['Feature_1'], row['Feature_2']]), axis=1, result_type='expand')
high_corr_pairs['sorted_pair'] = high_corr_pairs_sorted[0] + "_" + high_corr_pairs_sorted[1]
high_corr_pairs = high_corr_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

# 6. Display the results
print("Highly correlated feature pairs (|correlation| > 0.90):")
display(high_corr_pairs.sort_values(by='Correlation', ascending=False))

Highly correlated feature pairs (|correlation| > 0.90):


,Feature_1,Feature_2,Correlation
7,first_value_alanine_aminotransferase_(alt),first_value_asparate_aminotransferase_(ast),0.993
39,first_value_sodium,first_value_urea_nitrogen,0.989
5,first_value_absolute_lymphocyte_count,first_value_lymphocytes,0.988
24,first_value_glucose,first_value_sodium,0.985
25,first_value_glucose,first_value_urea_nitrogen,0.980
19,"first_value_calcium,_total",first_value_phosphate,0.979
10,first_value_alkaline_phosphatase,first_value_asparate_aminotransferase_(ast),0.979
6,first_value_alanine_aminotransferase_(alt),first_value_alkaline_phosphatase,0.974
22,first_value_creatinine,first_value_urea_nitrogen,0.971
11,first_value_alkaline_phosphatase,"first_value_bilirubin,_total",0.968


In [87]:
features_to_drop4 = [
    'first_value_alanine_aminotransferase_(alt)',
    'first_value_sodium',
    'first_value_absolute_lymphocyte_count',
    'first_value_glucose',
    'first_value_alkaline_phosphatase',
    'first_value_calcium,_total',
    'first_value_creatinine',
    'first_value_magnesium',
    'first_value_inr(pt)',
    'frequency_value_pt',
    'frequency_value_inr(pt)',
    'first_value_bilirubin,_total',
    'first_value_hematocrit',
    'abnormal_lab_count',
    'frequency_value_glucose',
    'frequency_value_ph',
    'frequency_value_ptt',
    'frequency_value_l',
    'first_value_immature_granulocytes',
]

df_cleaned_9.drop(columns=features_to_drop4, inplace=True)

In [88]:
#Checking the columns left which are highly correlated.
# 1. Compute the correlation matrix (numeric only)
corr_matrix = df_cleaned_9.corr(numeric_only=True)

# 2. Create a boolean mask for |correlation| > 0.90 and < 1.0 (exclude self-correlation)
high_corr_mask = (abs(corr_matrix) > 0.90) & (abs(corr_matrix) < 1.0)

# 3. Extract only the pairs of features that satisfy the condition
high_corr_pairs = high_corr_mask.stack().loc[lambda x: x].reset_index()
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

In [89]:
high_corr_pairs

,Feature_1,Feature_2,Correlation


In [90]:
df_cleaned_10 = df_cleaned_9.copy()

In [91]:
#USE FOR CASE 1 or CASE 3
#def drop_highly_correlated(df, threshold=0.90):
#    corr_matrix = df.corr().abs()
#    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
#    return df.drop(columns=to_drop), to_drop

#df_cleaned_10, dropped_cols = drop_highly_correlated(df_cleaned_9)

In [92]:
df_cleaned_10.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10839 entries, 0 to 10969
Data columns (total 122 columns):
 #    Column                                       Non-Null Count  Dtype  
---   ------                                       --------------  -----  
 0    subject_id                                   10839 non-null  int64  
 1    hadm_id                                      10839 non-null  int64  
 2    seq_num                                      10839 non-null  int64  
 3    class                                        10839 non-null  int64  
 4    gender                                       10839 non-null  int64  
 5    anchor_age                                   10839 non-null  int64  
 6    anchor_year                                  10839 non-null  int64  
 7    admission_type                               10839 non-null  int64  
 8    admission_location                           10839 non-null  int64  
 9    discharge_location                           10839 non-null  int

##Writing the files

In [93]:
df_cleaned_10.shape

(10839, 122)

In [94]:
df_cleaned_10['class'].value_counts()

,count
class,
3,2487
4,2466
2,2274
5,2198
1,1414


In [96]:
#df_cleaned_10.to_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/case_1_dataset.csv",index=False, header = True)
#df_cleaned_10.to_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/case_2_dataset.csv",index=False, header = True)
#df_cleaned_10.to_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/case_3_dataset.csv",index=False, header = True)